<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-700)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [3]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv')

#mypf = mypf[mypf['Nifty100'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()
stock_ids

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


array(['ACC', 'ADANIENT', 'ADANIGREEN', 'ADANIPORTS', 'ADANITRANS',
       'AMBUJACEM', 'APOLLOHOSP', 'ASIANPAINT', 'AUBANK', 'AUROPHARMA',
       'AXISBANK', 'BAJAJ-AUTO', 'BAJAJFINSV', 'BAJAJHLDNG', 'BAJFINANCE',
       'BANDHANBNK', 'BANKBARODA', 'BATAINDIA', 'BERGEPAINT',
       'BHARTIARTL', 'BIOCON', 'BOSCHLTD', 'BPCL', 'BRITANNIA', 'CANBK',
       'CHOLAFIN', 'CIPLA', 'COALINDIA', 'COLPAL', 'DABUR', 'DEEPAKNTR',
       'DIVISLAB', 'DLF', 'DMART', 'DRREDDY', 'EICHERMOT', 'EMAMILTD',
       'FEDERALBNK', 'GAIL', 'GILLETTE', 'GLAND', 'GODREJCP', 'GODREJIND',
       'GRASIM', 'HAVELLS', 'HCLTECH', 'HDFC', 'HDFCAMC', 'HDFCBANK',
       'HDFCLIFE', 'HEROMOTOCO', 'HINDALCO', 'HINDPETRO', 'HINDUNILVR',
       'IBREALEST', 'ICICIBANK', 'ICICIGI', 'ICICIPRULI', 'IGL',
       'INDIANB', 'INDIGO', 'INDIGOPNTS', 'INDUSINDBK', 'INDUSTOWER',
       'INFY', 'IOC', 'IRCTC', 'ITC', 'JINDALSTEL', 'JSWSTEEL',
       'JUBLFOOD', 'KOTAKBANK', 'LT', 'LTIM', 'LUPIN', 'M&M', 'MARICO',
       'MARUTI', '

In [4]:
new_stars = ['ADANIENT.NS', 'LTIM.NS', 'HCLTECH.NS', 'HDFC.NS', 'HDFCBANK.NS',
              'HINDUNILVR.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'IGL.NS', 'INDIANB.NS',
              'INDIGOPNTS.NS', 'ADANIGREEN.NS', 'INFY.NS', 'IRCTC.NS', 'ITC.NS',
              'JUBLFOOD.NS', 'M&M.NS', 'GODREJIND.NS', 'MARICO.NS', 'MCDOWELL-N.NS',
              'NTPC.NS', 'PAGEIND.NS', 'PIDILITIND.NS', 'PIIND.NS', 'RELIANCE.NS',
              'SBICARD.NS', 'SBIN.NS', 'TATACONSUM.NS', 'TATAPOWER.NS', 'TCS.NS',
              'TITAN.NS', 'UPL.NS', 'GRASIM.NS', 'INDUSINDBK.NS', 'BAJAJFINSV.NS',
              'CANBK.NS', 'CHOLAFIN.NS', 'CIPLA.NS', 'BERGEPAINT.NS', 'COALINDIA.NS',
              'AUBANK.NS', 'BAJAJ-AUTO.NS', 'DMART.NS', 'BRITANNIA.NS', 'DRREDDY.NS',
              'ASIANPAINT.NS', 'EICHERMOT.NS', 'BAJFINANCE.NS', 'BANDHANBNK.NS',
              'FEDERALBNK.NS', 'BHARTIARTL.NS']

curr_stars = ['ADANIENT.NS', 'LTIM.NS', 'HCLTECH.NS', 'HDFC.NS', 'HDFCBANK.NS',
               'HINDUNILVR.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'IGL.NS', 'INDIANB.NS',
               'INDIGOPNTS.NS', 'ADANIGREEN.NS', 'INFY.NS', 'IRCTC.NS', 'ITC.NS',
               'JUBLFOOD.NS', 'M&M.NS', 'GODREJIND.NS', 'MARICO.NS', 'MCDOWELL-N.NS',
               'NTPC.NS', 'PAGEIND.NS', 'PIDILITIND.NS', 'PIIND.NS', 'RELIANCE.NS',
               'SBICARD.NS', 'SBIN.NS', 'TATACONSUM.NS', 'TATAPOWER.NS', 'TCS.NS',
               'TITAN.NS', 'UPL.NS', 'GRASIM.NS', 'INDUSINDBK.NS', 'BAJAJFINSV.NS',
               'CANBK.NS', 'CHOLAFIN.NS', 'CIPLA.NS', 'BERGEPAINT.NS', 'COALINDIA.NS',
               'AUBANK.NS', 'BAJAJ-AUTO.NS', 'DMART.NS', 'BRITANNIA.NS', 'DRREDDY.NS',
               'ASIANPAINT.NS', 'EICHERMOT.NS', 'BAJFINANCE.NS', 'BANDHANBNK.NS',
               'FEDERALBNK.NS', 'BHARTIARTL.NS']

print('entries:', list(set(new_stars) - set(curr_stars)))
print('exits:', list(set(curr_stars) - set(new_stars)))

entries: []
exits: []


In [5]:
my_stocks = list(mypf[mypf['InPortfolio'] == 1]['Symbol'].values)

In [6]:
# update Nifty features
'''
stock = 'INFY'
column = 'Conviction'
value = 'H'
nifty100_df.loc[nifty100_df['Symbol'] == stock, column] = value

mypf.to_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv',index=False)
'''
mypf['Conviction'].value_counts()

L    67
H    31
M    27
Name: Conviction, dtype: int64

In [7]:
# stocks in portfolio
mypf['InPortfolio'].value_counts()

0    110
1     15
Name: InPortfolio, dtype: int64

In [8]:
# import libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [9]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_pe(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()  
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")        
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:
    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
  except ValueError as ve:
    current_pe = 1000 

  return current_pe


In [10]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20 
    mid_window = 50
    long_window = 150
    moving_avg = 'SMA'
    
    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='3y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_150'] = round((stock_df['Close'] - stock_df['150_SMA'])*100/stock_df['150_SMA'],2)
    stock_df['150_SMA_Lag10'] = stock_df['150_SMA'].shift(10)
    stock_df['Slope%_150'] = round((stock_df['150_SMA'] - stock_df['150_SMA_Lag10'])*100/stock_df['150_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)
    
    stock_df.ta.rsi(append=True)
    stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0)    
    #stock_df['RSI'] = round(stock_df['RSI'],0) 
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)   
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Upside%ATH'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','150_SMA','150_SMA_Lag10','50_SMA_Lag5'
                   ], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    if stock_id in my_stocks:
      current_pe = get_current_pe(stock_id)
    else:
      current_pe = 1000

    stock_id = stock_id.upper() + '.NS'    
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
    tmp['Med_PE'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]    
    tmp['Curr_PE'] = current_pe
    tmp['EstmtFlag'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['EstmtFlag'].values[0] 
    tmp['DuraFlag'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['DuraFlag'].values[0]   
    tmp['Conviction'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Conviction'].values[0]      
    df_prec_dev = df_prec_dev.append(tmp)

In [11]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,Stock,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH
Date,,,,,,,,,,
2023-03-28 00:00:00+05:30,16952.0,^NSEI,-4.78,-0.25,-0.52,36.445040,18812.0,15294.0,52.87,11.0
2023-04-06 00:00:00+05:30,17599.0,^NSEI,-1.12,-0.17,-0.38,59.607921,18812.0,15294.0,34.48,7.0
2023-04-17 00:00:00+05:30,17707.0,^NSEI,-0.54,0.00,-0.07,59.927005,18812.0,15294.0,31.41,6.0
2023-04-24 00:00:00+05:30,17743.0,^NSEI,-0.30,-0.02,-0.03,60.725021,18812.0,15294.0,30.39,6.0
2023-05-02 00:00:00+05:30,18148.0,^NSEI,1.94,-0.01,0.07,73.330854,18812.0,15294.0,18.87,4.0


In [12]:
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

mypf['Stock'] = mypf['Symbol'] + '.NS'
#df_prec_dev.drop(['Med_PE'], axis=1, inplace=True)

In [13]:
df_prec_dev['Discount%PE'] = round((df_prec_dev['Med_PE']-df_prec_dev['Curr_PE'])*100/(df_prec_dev['Med_PE']),0)
df_prec_dev = df_prec_dev.set_index('Stock')

In [14]:
# portfolio stocks

tmp_df = df_prec_dev[df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 15


,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,Avg_Cost,Shares,Med_PE,Curr_PE,EstmtFlag,DuraFlag,Conviction,Discount%PE
Stock,,,,,,,,,,,,,,,,,
INDIGOPNTS.NS,1158.0,-7.67,-2.31,0.09,66.991285,2673.0,984.0,89.70,131.0,1152.30,44.0,104.9,46.8,1.0,1.0,H,55.0
JUBLFOOD.NS,462.0,-9.49,-2.35,0.00,64.241684,903.0,422.0,91.68,95.0,439.80,68.0,90.5,72.3,1.0,1.0,H,20.0
IRCTC.NS,627.0,-5.01,-0.91,-0.33,68.295579,1164.0,375.0,68.06,86.0,609.75,42.0,77.4,53.3,1.0,1.0,H,31.0
DMART.NS,3546.0,-7.15,-1.62,0.06,60.074933,5324.0,3045.0,78.02,50.0,3476.95,9.0,160.1,98.1,1.0,1.0,H,39.0
INFY.NS,1277.0,-14.03,-0.94,-2.40,38.962566,1898.0,1223.0,92.00,49.0,1240.70,24.0,29.0,22.0,1.0,1.0,H,24.0
BATAINDIA.NS,1491.0,-7.10,-1.74,-0.28,68.192589,2173.0,1387.0,86.77,46.0,1483.40,14.0,126.0,60.1,1.0,1.0,H,52.0
GILLETTE.NS,4474.0,-7.65,-1.36,-1.07,58.896212,5944.0,4186.0,83.62,33.0,4353.00,7.0,61.2,49.1,0.0,1.0,M,20.0
PAGEIND.NS,40229.0,-7.01,-1.39,0.48,61.768640,53631.0,29040.0,54.50,33.0,38201.00,1.0,97.7,65.6,1.0,1.0,H,33.0
BAJFINANCE.NS,6249.0,-3.14,-1.35,-0.30,66.893248,7901.0,5265.0,62.67,26.0,6044.41,8.0,52.6,32.9,1.0,1.0,H,37.0


In [15]:
tmp_df.sort_values(by = 'Discount%PE', ascending=False)

,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,Avg_Cost,Shares,Med_PE,Curr_PE,EstmtFlag,DuraFlag,Conviction,Discount%PE
Stock,,,,,,,,,,,,,,,,,
INDIGOPNTS.NS,1158.0,-7.67,-2.31,0.09,66.991285,2673.0,984.0,89.70,131.0,1152.30,44.0,104.9,46.8,1.0,1.0,H,55.0
BATAINDIA.NS,1491.0,-7.10,-1.74,-0.28,68.192589,2173.0,1387.0,86.77,46.0,1483.40,14.0,126.0,60.1,1.0,1.0,H,52.0
DMART.NS,3546.0,-7.15,-1.62,0.06,60.074933,5324.0,3045.0,78.02,50.0,3476.95,9.0,160.1,98.1,1.0,1.0,H,39.0
BAJFINANCE.NS,6249.0,-3.14,-1.35,-0.30,66.893248,7901.0,5265.0,62.67,26.0,6044.41,8.0,52.6,32.9,1.0,1.0,H,37.0
PAGEIND.NS,40229.0,-7.01,-1.39,0.48,61.768640,53631.0,29040.0,54.50,33.0,38201.00,1.0,97.7,65.6,1.0,1.0,H,33.0
IRCTC.NS,627.0,-5.01,-0.91,-0.33,68.295579,1164.0,375.0,68.06,86.0,609.75,42.0,77.4,53.3,1.0,1.0,H,31.0
INFY.NS,1277.0,-14.03,-0.94,-2.40,38.962566,1898.0,1223.0,92.00,49.0,1240.70,24.0,29.0,22.0,1.0,1.0,H,24.0
EICHERMOT.NS,3356.0,1.46,-0.54,0.10,69.656577,3850.0,2241.0,30.70,15.0,3167.25,6.0,44.7,35.0,1.0,1.0,H,22.0
GILLETTE.NS,4474.0,-7.65,-1.36,-1.07,58.896212,5944.0,4186.0,83.62,33.0,4353.00,7.0,61.2,49.1,0.0,1.0,M,20.0


In [16]:
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,Avg_Cost,Shares,Med_PE,Curr_PE,EstmtFlag,DuraFlag,Conviction,Discount%PE
Stock,,,,,,,,,,,,,,,,,
PAGEIND.NS,40229.0,-7.01,-1.39,0.48,61.768640,53631.0,29040.0,54.50,33.0,38201.00,1.0,97.7,65.6,1.0,1.0,H,33.0
ASIANPAINT.NS,2900.0,-3.15,-1.14,0.43,60.544903,3566.0,2576.0,67.27,23.0,2856.73,17.0,87.9,72.7,1.0,1.0,H,17.0
TATACONSUM.NS,780.0,3.53,-0.80,0.42,76.189925,875.0,655.0,43.18,12.0,702.36,40.0,68.8,66.1,1.0,1.0,H,4.0
PIDILITIND.NS,2420.0,-3.48,-1.16,0.38,58.956100,2902.0,1991.0,52.91,20.0,2419.95,8.0,94.6,98.8,1.0,1.0,H,-4.0
RELIANCE.NS,2441.0,-0.32,-0.49,0.26,67.817505,2811.0,2029.0,47.31,15.0,2353.60,13.0,29.1,24.7,1.0,0.0,H,15.0
HDFCBANK.NS,1687.0,6.38,0.76,0.12,62.501569,1696.0,1281.0,2.17,1.0,1672.84,30.0,22.0,20.4,1.0,0.0,H,7.0
EICHERMOT.NS,3356.0,1.46,-0.54,0.10,69.656577,3850.0,2241.0,30.70,15.0,3167.25,6.0,44.7,35.0,1.0,1.0,H,22.0
INDIGOPNTS.NS,1158.0,-7.67,-2.31,0.09,66.991285,2673.0,984.0,89.70,131.0,1152.30,44.0,104.9,46.8,1.0,1.0,H,55.0
DMART.NS,3546.0,-7.15,-1.62,0.06,60.074933,5324.0,3045.0,78.02,50.0,3476.95,9.0,160.1,98.1,1.0,1.0,H,39.0


In [17]:
df_prec_dev.drop([ 'Avg_Cost', 'Shares', 'Med_PE', 'Curr_PE', 'Discount%PE'], axis=1, inplace=True)

In [18]:
# all time high sales and profit TTM

star_stocks = new_stars

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 9


,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction
Stock,,,,,,,,,,,,
ADANIENT.NS,1920.0,-34.23,-4.01,-0.06,57.929937,4165.0,1194.0,75.56,117.0,1.0,0.0,M
LTIM.NS,4548.0,-0.43,-0.39,-0.71,54.270831,7439.0,3735.0,78.05,64.0,1.0,0.0,M
BANDHANBNK.NS,232.0,-1.74,-2.12,-0.46,68.754988,346.0,187.0,71.70,49.0,1.0,1.0,L
ICICIGI.NS,1087.0,-5.39,-0.87,-0.55,51.356903,1620.0,1060.0,95.18,49.0,1.0,0.0,M
SBICARD.NS,782.0,-0.77,-1.52,0.40,62.386557,1133.0,673.0,76.30,45.0,1.0,0.0,M
TATAPOWER.NS,202.0,-4.38,-1.42,-0.50,59.303026,287.0,103.0,46.20,42.0,1.0,0.0,M
BERGEPAINT.NS,615.0,4.22,-0.68,0.69,67.362213,858.0,540.0,76.42,40.0,1.0,0.0,M
CIPLA.NS,918.0,-11.33,-0.97,-1.32,54.392294,1173.0,856.0,80.44,28.0,1.0,0.0,M
TCS.NS,3219.0,-0.42,0.22,-0.95,53.131158,3886.0,2915.0,68.69,21.0,1.0,0.0,M


In [19]:
# star stocks ordered based on slope
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction
Stock,,,,,,,,,,,,
BERGEPAINT.NS,615.0,4.22,-0.68,0.69,67.362213,858.0,540.0,76.42,40.0,1.0,0.0,M
SBICARD.NS,782.0,-0.77,-1.52,0.40,62.386557,1133.0,673.0,76.30,45.0,1.0,0.0,M
ADANIENT.NS,1920.0,-34.23,-4.01,-0.06,57.929937,4165.0,1194.0,75.56,117.0,1.0,0.0,M
BANDHANBNK.NS,232.0,-1.74,-2.12,-0.46,68.754988,346.0,187.0,71.70,49.0,1.0,1.0,L
TATAPOWER.NS,202.0,-4.38,-1.42,-0.50,59.303026,287.0,103.0,46.20,42.0,1.0,0.0,M
ICICIGI.NS,1087.0,-5.39,-0.87,-0.55,51.356903,1620.0,1060.0,95.18,49.0,1.0,0.0,M
LTIM.NS,4548.0,-0.43,-0.39,-0.71,54.270831,7439.0,3735.0,78.05,64.0,1.0,0.0,M
TCS.NS,3219.0,-0.42,0.22,-0.95,53.131158,3886.0,2915.0,68.69,21.0,1.0,0.0,M
CIPLA.NS,918.0,-11.33,-0.97,-1.32,54.392294,1173.0,856.0,80.44,28.0,1.0,0.0,M


In [20]:
# stocks excluded from all time high

excluded_stocks = ['ACC.NS','AMBUJACEM.NS','CADILAHC.NS','GILLETTE.NS','NESTLEIND.NS','PGHH.NS','SIEMENS.NS']

tmp_df = df_prec_dev[df_prec_dev.index.isin(excluded_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
#tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 5


,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction
Stock,,,,,,,,,,,,
ACC.NS,1764.0,-18.55,-2.59,-0.97,52.863947,2748.0,1614.0,86.77,56.0,0.0,0.0,L
AMBUJACEM.NS,394.0,-15.73,-1.71,0.81,58.943130,588.0,275.0,61.98,49.0,0.0,0.0,L
PGHH.NS,13895.0,0.20,-0.19,0.20,48.597138,15778.0,12000.0,49.84,14.0,0.0,0.0,L
NESTLEIND.NS,21746.0,11.86,0.75,1.22,81.247496,21761.0,15919.0,0.26,0.0,1.0,1.0,H
SIEMENS.NS,3500.0,16.28,0.83,0.94,70.687663,3500.0,1922.0,0.00,0.0,1.0,1.0,H


In [21]:
# prospects with heavy upside
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 2


,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction
Stock,,,,,,,,,,,,
INDUSTOWER.NS,156.0,-10.60,-2.29,-0.65,68.468127,303.0,137.0,88.55,94.0,1.0,1.0,H
BANDHANBNK.NS,232.0,-1.74,-2.12,-0.46,68.754988,346.0,187.0,71.70,49.0,1.0,1.0,L


In [22]:
# prospects near 52W low
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)]
tmp_df = tmp_df[tmp_df['Discount%'] > 60]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=False)

qualified stocks: 3


,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction
Stock,,,,,,,,,,,,
INDUSTOWER.NS,156.0,-10.60,-2.29,-0.65,68.468127,303.0,137.0,88.55,94.0,1.0,1.0,H
HDFCLIFE.NS,544.0,1.21,-0.56,0.60,60.707617,757.0,461.0,71.96,39.0,1.0,1.0,H
BANDHANBNK.NS,232.0,-1.74,-2.12,-0.46,68.754988,346.0,187.0,71.70,49.0,1.0,1.0,L


In [23]:
# prospects near 52W high
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)]
tmp_df = tmp_df[tmp_df['Discount%'] < 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=True)

qualified stocks: 7


,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction
Stock,,,,,,,,,,,,
INDIANB.NS,329.0,21.91,2.59,1.05,67.806211,329.0,112.0,0.00,0.0,1.0,1.0,H
SIEMENS.NS,3500.0,16.28,0.83,0.94,70.687663,3500.0,1922.0,0.00,0.0,1.0,1.0,H
NESTLEIND.NS,21746.0,11.86,0.75,1.22,81.247496,21761.0,15919.0,0.26,0.0,1.0,1.0,H
BANKBARODA.NS,188.0,14.00,1.82,0.60,74.425800,193.0,71.0,4.10,3.0,1.0,1.0,H
CANBK.NS,320.0,9.59,1.03,0.69,73.098424,337.0,139.0,8.59,5.0,1.0,1.0,H
TITAN.NS,2664.0,4.71,-0.08,0.72,69.021698,2770.0,1576.0,8.88,4.0,1.0,1.0,H
ICICIBANK.NS,922.0,4.31,0.00,0.69,69.562766,953.0,619.0,9.28,3.0,1.0,1.0,H


In [24]:
# prospects in bull run
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)]
tmp_df = tmp_df[(tmp_df['Slope%_150'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_150'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 2


,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction
Stock,,,,,,,,,,,,
NESTLEIND.NS,21746.0,11.86,0.75,1.22,81.247496,21761.0,15919.0,0.26,0.0,1.0,1.0,H
SIEMENS.NS,3500.0,16.28,0.83,0.94,70.687663,3500.0,1922.0,0.00,0.0,1.0,1.0,H


In [25]:
# prospects about to turn around
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)]
tmp_df = tmp_df[(tmp_df['Slope%_150'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_150'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 6


,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction
Stock,,,,,,,,,,,,
TITAN.NS,2664.0,4.71,-0.08,0.72,69.021698,2770.0,1576.0,8.88,4.0,1.0,1.0,H
HDFCLIFE.NS,544.0,1.21,-0.56,0.60,60.707617,757.0,461.0,71.96,39.0,1.0,1.0,H
INDUSINDBK.NS,1151.0,0.17,-0.26,-0.09,62.562701,1264.0,771.0,22.92,10.0,1.0,1.0,H
MARUTI.NS,8777.0,0.98,-0.32,-0.27,62.411979,9528.0,6565.0,25.35,9.0,1.0,1.0,H
BANDHANBNK.NS,232.0,-1.74,-2.12,-0.46,68.754988,346.0,187.0,71.70,49.0,1.0,1.0,L
INDUSTOWER.NS,156.0,-10.60,-2.29,-0.65,68.468127,303.0,137.0,88.55,94.0,1.0,1.0,H


In [26]:
# cross-check the features
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] != 1) | (tmp_df['DuraFlag'] != 1)]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 41


,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction
Stock,,,,,,,,,,,,
ADANITRANS.NS,1022.0,-52.95,-9.21,-2.35,50.872484,4105.0,643.0,89.05,302.0,1.0,-1.0,L
PEL.NS,742.0,-7.99,-2.23,-1.77,62.608865,2866.0,635.0,95.20,286.0,-1.0,0.0,L
GLAND.NS,1376.0,-12.82,-4.25,0.31,59.927487,4316.0,1154.0,92.98,214.0,1.0,0.0,M
ADANIGREEN.NS,975.0,-36.87,-6.09,2.31,60.192576,2970.0,462.0,79.55,205.0,-1.0,-1.0,L
IBREALEST.NS,73.0,3.17,-1.41,1.72,75.232128,191.0,47.0,81.94,162.0,-1.0,-1.0,L
ADANIENT.NS,1920.0,-34.23,-4.01,-0.06,57.929937,4165.0,1194.0,75.56,117.0,1.0,0.0,M
NETWORK18.NS,58.0,-5.81,-1.61,0.00,57.749552,114.0,40.0,75.68,97.0,-1.0,-1.0,L
WHIRLPOOL.NS,1317.0,-8.93,-2.01,0.30,45.713373,2510.0,1254.0,94.98,91.0,1.0,0.0,M
ZEEL.NS,196.0,-16.45,-1.70,-1.48,44.421672,367.0,164.0,84.24,87.0,-1.0,1.0,L


In [27]:
stock_id = 'MUTHOOTFIN.NS'
df_prec_dev[df_prec_dev.index == stock_id]

,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction
Stock,,,,,,,,,,,,
MUTHOOTFIN.NS,1027.0,1.22,0.1,0.31,63.334678,1651.0,897.0,82.76,61.0,-1.0,0.0,L
